In [99]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [101]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

# 전용면적당 법정주차대수 

## 법정주차대수 Column 추가

In [102]:
train.insert(loc=14, column='법정주차대수', value=0.0)

## 전용면적을 지역별 각 지역 기준으로 나누기

In [103]:
# for문 columns 설정
Gwang_colName = ['부산광역시', '대전광역시', '광주광역시', '울산광역시', '대구광역시', '세종특별자치시', '경기도']
Other_colName = ['경상북도', '경상남도', '전라북도', '전라남도', '강원도', '충청남도', '충청북도', '제주특별자치도']

# Train
# 1. 서울특별시 
# - 전용면적 {85이하 - 75} | {85초과 - 65} 나누기
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 85) ].index
train.loc[idx, '법정주차대수'] = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 85) ]['전용면적'] / 75

idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 85) ].index
train.loc[idx, '법정주차대수'] = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 85) ]['전용면적'] / 65

# 2. 광역시 + 경기도
# - 전용면적 {85이하 - 85} | {85초과 - 70} 나누기
for colName in Gwang_colName:
    idx = train[ (train['지역'] == colName) & (train['전용면적'] <= 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] <= 85) ]['전용면적'] / 85
    idx = train[(train['지역'] == colName) & (train['전용면적'] > 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] > 85) ]['전용면적'] / 70
    
# 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 전용면적 {85이하 - 95} | {85초과 - 75} 나누기
for colName in Other_colName:
    idx = train[ (train['지역'] == colName) & (train['전용면적'] <= 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] <= 85) ]['전용면적'] / 95
    idx = train[(train['지역'] == colName) & (train['전용면적'] > 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] > 85) ]['전용면적'] / 75
    
# Test
# 2. 광역시 + 경기도
# - 전용면적 {85이하 - 85} | {85초과 - 70} 나누기
for colName in Gwang_colName:
    idx = test[ (test['지역'] == colName) & (test['전용면적'] <= 85) ].index
    test.loc[idx, '법정주차대수'] = test[(test['지역'] == colName) & (test['전용면적'] <= 85) ]['전용면적'] / 85
    idx = test[(test['지역'] == colName) & (test['전용면적'] > 85) ].index
    test.loc[idx, '법정주차대수'] = test[(test['지역'] == colName) & (test['전용면적'] > 85) ]['전용면적'] / 70
    
# 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 전용면적 {85이하 - 95} | {85초과 - 75} 나누기
for colName in Other_colName:
    idx = test[ (test['지역'] == colName) & (test['전용면적'] <= 85) ].index
    test.loc[idx, '법정주차대수'] = test[(test['지역'] == colName) & (test['전용면적'] <= 85) ]['전용면적'] / 95
    idx = test[(test['지역'] == colName) & (test['전용면적'] > 85) ].index
    test.loc[idx, '법정주차대수'] = test[(test['지역'] == colName) & (test['전용면적'] > 85) ]['전용면적'] / 75

## 법정주차대수 소수점 정리

In [104]:
# Train
# 1. 서울특별시 
# 1) 전용면적 30이하 : 0.5 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 30) ].index
train.loc[idx, '법정주차대수'] = 0.5

# 2) 전용면적 30초과 ~ 60이하 : 0.8 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 30) & (train['전용면적'] <= 60) ].index
train.loc[idx, '법정주차대수'] = 0.8

# 3) 전용면적 60초과 : 1.xxx -> 2 or 2.xxxx -> 3
# - 나머지가 조금이라도 나오면 몫의 값 + 1 => np.ceil 함수 사용
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ].index
train.loc[idx, '법정주차대수'] = np.ceil(train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ]['법정주차대수'])

# 2. 광역시 + 경기도
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Gwang_colName:
    idx = train[ train['지역'] == colName ].index
    train.loc[idx, '법정주차대수'] = np.ceil(train[ train['지역'] == colName ]['법정주차대수'])

# 3. 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Other_colName:
    idx = train[ train['지역'] == colName ].index
    train.loc[idx, '법정주차대수'] = np.ceil(train[ train['지역'] == colName ]['법정주차대수'])
    
# Test
# 2. 광역시 + 경기도
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Gwang_colName:
    idx = test[ test['지역'] == colName ].index
    test.loc[idx, '법정주차대수'] = np.ceil(test[ test['지역'] == colName ]['법정주차대수'])

# 3. 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Other_colName:
    idx = test[ test['지역'] == colName ].index
    test.loc[idx, '법정주차대수'] = np.ceil(test[ test['지역'] == colName ]['법정주차대수'])

# 컬럼명 바꾸기

In [105]:
train = train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

test = test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

# 결측치 처리

In [106]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
법정주차대수        0
등록차량수         0
dtype: int64
-------- test.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
법정주차대수        0
dtype: int64


In [107]:
# train 데이터
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)
train['지하철'] = train['지하철'].fillna(0)
train['버스'] = train['버스'].fillna(0)

# test 데이터
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

In [108]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
법정주차대수      0
등록차량수       0
dtype: int64
-------- test.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
법정주차대수      0
dtype: int64


# '-' 데이터 처리

## 장기전세 임대료 데이터 처리

In [109]:
idx = train[train['공급유형'] == '장기전세'].index
train.loc[idx, '임대료'] = 0.0

## 자격유형 'H' & 임대료 '-' 데이터 처리

In [110]:
rental_cost_mean = []
colletion_range = 0.5 # 수집할 +/- 범위

# 자격유형 'H'이면서  임대료 '-'인 것들의 전용면적 가져오기
area_list = train[(train['임대료']=='-')&(train['자격유형']=='H')]['전용면적'].to_list()

# 조건문
# 국민임대 & 'H' & '-'  & 면적-0.5 이상 & 면적+0.5 이하 
for area in area_list:
    rental_cost_mean.append( np.mean(train[ (train['공급유형'] == '국민임대') & (train['자격유형'] == 'H') & (train['임대료'] != '-') 
      & (train['전용면적'] >= (area-colletion_range)) & (train['전용면적'] <= (area+colletion_range)) ]['임대료'].values.astype(float)) )

# 데이터 입력
idx = train[(train['임대료']=='-')&(train['자격유형']=='H')].index
train.loc[idx, '임대료'] = rental_cost_mean

## '-' 데이터 처리

In [111]:
# 임대료, 임대보증금 데이터에 - 값이 있어서 이를 0으로 변환
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)

# 지하철 Y or N로 변경

In [112]:
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

# 임대건물구분 컬럼 처리

In [113]:
# 임대건물구분에서 7개의 아파트는 상가로 예상이됨 이를 처리 함.
# 임대건물구분이 상가인 경우는 전용면적별세대수도 1이므로 이를 처리 함.

idx = train[(train['임대건물구분']=='아파트') & (train['자격유형']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

# 지역 데이터를 숫자로 매핑

In [114]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

# 전용면적을 상/하한 적용

In [115]:
# 전용면적을 5의 배수로 변경 후 상/하한 적용
# 하한 값 15, 상한 값 100을 적용

train['전용면적'] = np.round(train['전용면적'], 0)
train['전용면적'] = train['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

test['전용면적'] = np.round(test['전용면적'], 0)
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 법정주차수요 만들기

In [116]:
value = train['총세대수'] * train['법정주차대수']
train.insert(loc=15, column='법정주차수요', value=value)

value = test['총세대수'] * test['법정주차대수']
test['법정주차수요'] = value

# 아파트 / 상가 데이터 분리

In [34]:
# Train
apart_train = train[train['임대건물구분'] == '아파트']
shop_train = train[train['임대건물구분'] == '상가']

# Test
apart_test = test[test['임대건물구분'] == '아파트']
shop_test = test[test['임대건물구분'] == '상가']

# 상관관계

In [ ]:
pd.DataFrame(apart_train.corr()['등록차량수'].abs().sort_values(ascending=False))

# 학습 데이터 생성

In [35]:
# Train
apart_train_data = apart_train[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수','등록차량수']]
shop_train_data = shop_train[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수','등록차량수']]

# Test
apart_test_data = apart_test[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수']]
shop_test_data = shop_test[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수']]

print("Apart train 데이터:",apart_train_data.shape)
print("Shop train 데이터:",shop_train_data.shape)
print("Apart test 데이터:",apart_test_data.shape)
print("Shop test 데이터:",shop_test_data.shape)

Apart train 데이터: (2383, 8)
Shop train 데이터: (569, 8)
Apart test 데이터: (845, 7)
Shop test 데이터: (177, 7)


In [36]:
# 데이터 target 설정
x_apart_train = apart_train_data.iloc[:,:7]
y_apart_train = apart_train_data.iloc[:,-1]

x_shop_train = shop_train_data.iloc[:,:7]
y_shop_train = shop_train_data.iloc[:,-1]

x_apart_test = apart_test_data
x_shop_test = shop_test_data

print("x_apart_train 데이터:",x_apart_train.shape)
print("y_apart_train 데이터:",y_apart_train.shape)
print("x_shop_train 데이터:",x_shop_train.shape)
print("y_shop_train 데이터:",y_shop_train.shape)
print("x_apart_test 데이터:",x_apart_test.shape)
print("x_shop_test 데이터:",x_shop_test.shape)

x_apart_train 데이터: (2383, 7)
y_apart_train 데이터: (2383,)
x_shop_train 데이터: (569, 7)
y_shop_train 데이터: (569,)
x_apart_test 데이터: (845, 7)
x_shop_test 데이터: (177, 7)


# 모델링

In [18]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1, random_state=42)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

In [19]:
import xgboost

xgb = xgboost.XGBRegressor()

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

[21:58:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# 예측 데이터셋 생성

In [20]:
df = []
df = pd.DataFrame(df)
df['code'] = test['단지코드']
df['num'] = pred
sub = df.groupby(['code'],sort=False,as_index=False).mean()

In [21]:
# sub.to_csv('xgb.csv', index=False)
sub.head()

,code,num
0,C1072,696.358887
1,C1128,1199.364014
2,C1456,601.295471
3,C1840,557.970764
4,C1332,1090.653198


# 모델 평가

In [22]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(random_state=0)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

mae : 28.853726287262873
r2_score : 0.9783652648951725


In [23]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

import xgboost

xgb = xgboost.XGBRegressor(random_state=0)

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

[21:58:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
mae : 86.0129791818014
r2_score : 0.9169686194635103
